In [47]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm 
import getFamaFrenchFactors as gff 
import datetime as dt
import multiprocessing

ticker = 'V'
start = '2000-01-01'
end = '2019-01-01'

stock_data = yf.download(ticker, start, end, actions=True)
print(stock_data)


[*********************100%%**********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2008-03-19   14.875000   17.250000   13.750000   14.125000   12.652096   
2008-03-20   14.600000   16.250000   14.375000   16.087500   14.409959   
2008-03-24   16.809999   16.924999   14.750000   14.932500   13.375390   
2008-03-25   15.145000   16.062500   14.955000   15.812500   14.163630   
2008-03-26   15.682500   16.120001   15.392500   15.990000   14.322618   
...                ...         ...         ...         ...         ...   
2018-12-24  122.080002  125.209999  121.599998  121.730003  117.648727   
2018-12-26  123.019997  130.240005  123.000000  130.229996  125.863731   
2018-12-27  127.989998  132.009995  126.519997  132.009995  127.584084   
2018-12-28  133.080002  133.619995  129.660004  130.940002  126.549927   
2018-12-31  131.679993  132.440002  130.229

In [48]:
ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors" : 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)
print(ff3_monthly)


            Mkt-RF     SMB     HML      RF
Date                                      
1926-07-31  0.0296 -0.0256 -0.0243  0.0022
1926-08-31  0.0264 -0.0117  0.0382  0.0025
1926-09-30  0.0036 -0.0140  0.0013  0.0023
1926-10-31 -0.0324 -0.0009  0.0070  0.0032
1926-11-30  0.0253 -0.0010 -0.0051  0.0031
...            ...     ...     ...     ...
2023-08-31 -0.0239 -0.0316 -0.0106  0.0045
2023-09-30 -0.0524 -0.0251  0.0152  0.0043
2023-10-31 -0.0319 -0.0387  0.0019  0.0047
2023-11-30  0.0884 -0.0002  0.0164  0.0044
2023-12-31  0.0485  0.0635  0.0494  0.0043

[1170 rows x 4 columns]


In [49]:
import yfinance as yf
import multiprocessing
ticker = ticker
start_test = '2020-01-01'
end_test = '2021-01-01'


# Download historical stock data
stock_test = yf.download(ticker, start_test, end_test, actions=True)

# Drop unnecessary columns
columns_to_drop = ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
stock_test.drop(columns=columns_to_drop, inplace=True)

# Extract adjusted closing prices
adj_close_prices = stock_test['Adj Close']
global percentage_difference
# Calculate percentage difference from the start of the year to the end
start_of_year_price = adj_close_prices.iloc[0]
end_of_year_price = adj_close_prices.iloc[-1]
percentage_difference = ((end_of_year_price - start_of_year_price) / start_of_year_price) * 100

print(stock_test)
print(f"Start of Year Date: {start_test}, Adjusted Close: ${start_of_year_price:.2f}")
print(f"End of Year Date: {end_test}, Adjusted Close: ${end_of_year_price:.2f}")
print(f"Percentage Difference: {percentage_difference:.2f}%")


[*********************100%%**********************]  1 of 1 completed
             Adj Close
Date                  
2020-01-02  185.897491
2020-01-03  184.419006
2020-01-06  184.020248
2020-01-07  183.533905
2020-01-08  186.675598
...                ...
2020-12-24  204.246628
2020-12-28  208.092743
2020-12-29  209.795593
2020-12-30  213.700470
2020-12-31  214.062561

[253 rows x 1 columns]
Start of Year Date: 2020-01-01, Adjusted Close: $185.90
End of Year Date: 2021-01-01, Adjusted Close: $214.06
Percentage Difference: 15.15%


In [50]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff
import multiprocessing
import matplotlib.pyplot as plt
import datetime as dt


##ALLOW USER INPUT##
ticker = ticker
#trainging data#
start_train = '2001-01-01'
end_train ='2020-01-01'



#testing data
test_start = '2017-01-01'
test_end= '2018-01-01'


stock_data = yf.download(ticker, start_train, end_train)
#
test_data=yf.download(ticker, test_start, test_end)


ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)

#
ff3_test_monthly=gff.famaFrench3Factor(frequency='m')
ff3_test_monthly.rename(columns={"date_ff_factors" : 'Date'}, inplace=True)
ff3_test_monthly.set_index('Date', inplace=True)

stock_returns = stock_data['Adj Close'].resample('m').last().pct_change().dropna()

stock_returns.name = "Month_Rtn"
ff_data = ff3_monthly.merge(stock_returns,on='Date')
 
X = ff_data[['Mkt-RF', 'SMB', 'HML']]

##Excess return on a monthly basis
y = ff_data['Month_Rtn'] - ff_data['RF']

X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

rf = ff_data['RF'].mean()
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium = ff3_monthly['SMB'].mean()
value_premium = ff3_monthly['HML'].mean()
global expected_yearly_return_int
expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 
expected_yearly_return = expected_monthly_return * 12
expected_yearly_return_int = float (expected_yearly_return *100)
print("Expected yearly return: " + str(expected_yearly_return))
print (f"Number {expected_yearly_return_int}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.387
Method:                 Least Squares   F-statistic:                     30.51
Date:                Thu, 01 Feb 2024   Prob (F-statistic):           3.59e-15
Time:                        19:00:01   Log-Likelihood:                 225.26
No. Observations:                 141   AIC:                            -442.5
Df Residuals:                     137   BIC:                            -430.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      

In [51]:
import multiprocessing 
#Just to give more weight on newer data
avg_est_return = ((expected_yearly_return_int + percentage_difference * 1.5 )) / 2 
print(avg_est_return)

13.725740321921247


In [52]:
#get past 7 days of closing price 

import yfinance as yf
import pandas as pd
import multiprocessing
import time 
import datetime as dt 

current_time = dt.datetime.now()
# Download historical stock data
ticker = ticker
start_date = '2024-01-01'
end_date = '2024-01-07'

stock_data = yf.download(ticker, start=start_date, end=end_date)

# Convert the historical data to a DataFrame
df = pd.DataFrame(stock_data) 
print(current_time)

[*********************100%%**********************]  1 of 1 completed
2024-02-05 13:05:34.959604
